# Classification

Cassification is the process of dividing the dataset into different categories or groups by adding labels.

# Naive Bayes

## 手工

In [281]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import os

In [282]:
data = pd.read_csv(os.path.join(os.path.dirname(os.getcwd()), 'dataset\\titanic-dataset\\titanic_train.csv'))

X = data.pclass.to_numpy().reshape(-1, 1)
Y = data.survived.to_numpy()

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [283]:
p_c1 = np.mean(Y_train)
p_c0 = 1 - p_c1

In [284]:
X_c0 = X_train[Y_train==0]
X_c1 = X_train[Y_train==1]

In [285]:
p_1_c0 = np.mean(X_c0==1)
p_2_c0 = np.mean(X_c0==2)
p_3_c0 = np.mean(X_c0==3)

In [286]:
p_1_c0,p_2_c0, p_3_c0

(0.15402298850574714, 0.1793103448275862, 0.6666666666666666)

In [287]:
p_1_c1 = np.mean(X_c1==1)
p_2_c1 = np.mean(X_c1==2)
p_3_c1 = np.mean(X_c1==3)

In [288]:
p_1_c1,p_2_c1, p_3_c1

(0.4122448979591837, 0.22857142857142856, 0.35918367346938773)

In [289]:
p_c0_1 = p_c0 * p_1_c0 / (p_c0 * p_1_c0 + p_c1 * p_1_c1)
p_c0_2 = p_c0 * p_2_c0 / (p_c0 * p_2_c0 + p_c1 * p_2_c1)
p_c0_3 = p_c0 * p_3_c0 / (p_c0 * p_3_c0 + p_c1 * p_3_c1)

In [290]:
p_c0_1, p_c0_2, p_c0_3

(0.3988095238095238, 0.582089552238806, 0.7671957671957672)

In [291]:
p_c1_1 = p_c1 * p_1_c1 / (p_c0 * p_1_c0 + p_c1 * p_1_c1)
p_c1_2 = p_c1 * p_2_c1 / (p_c0 * p_2_c0 + p_c1 * p_2_c1)
p_c1_3 = p_c1 * p_3_c1 / (p_c0 * p_3_c0 + p_c1 * p_3_c1)

In [292]:
p_c1_1, p_c1_2, p_c1_3

(0.6011904761904762, 0.417910447761194, 0.23280423280423282)

In [293]:
def is_survived(pclass):
    if pclass == 1:
        return 1
    else:
        return 0

In [294]:
Y_predict = []
for pclass in X_test:
    Y_predict.append(is_survived(pclass))
Y_predict = np.array(Y_predict)

In [295]:
counter = 0
for i, j in zip(Y_predict, Y_test):
    if i == j:
        counter += 1 
        
accuracy = counter / len(Y_test)
accuracy

0.6588235294117647

## 自动

In [296]:
# import some preprocessing methods, such as train/test split and feature encoding, accuracy checker
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import accuracy_score

from sklearn.naive_bayes import CategoricalNB

clf = CategoricalNB(alpha=1.0e-10)
clf.fit(X_train, Y_train)

print('Probabilities p(c0) and p(c1):\n', [np.exp(x) for x in clf.class_log_prior_.tolist()])
print('Probabilities p(1|c0), p(2|c0), p(3|c0):\n', [np.exp(x) for x in clf.feature_log_prob_[0][0].tolist()])
print('Probabilities p(1|c1), p(2|c1), p(3|c1):\n', [np.exp(x) for x in clf.feature_log_prob_[0][1].tolist()])

Probabilities p(c0) and p(c1):
 [0.6397058823529415, 0.3602941176470591]
Probabilities p(1|c0), p(2|c0), p(3|c0):
 [2.2988505747105254e-13, 0.15402298850583543, 0.17931034482765132, 0.6666666666662834]
Probabilities p(1|c1), p(2|c1), p(3|c1):
 [4.0816326530545546e-13, 0.4122448979589189, 0.22857142857146348, 0.35918367346920943]


In [297]:
Y_predict = clf.predict(X_test)
predictions = pd.DataFrame({'class': X_test.flatten(), 'predict_proba': clf.predict_proba(X_test)[:,1],
                            'predict': Y_predict, 'survived': Y_test})

print(predictions)

print('Accuracy:', accuracy_score(Y_predict, Y_test))

     class  predict_proba  predict  survived
0        3       0.232804        0         0
1        3       0.232804        0         0
2        3       0.232804        0         0
3        3       0.232804        0         0
4        3       0.232804        0         0
..     ...            ...      ...       ...
165      3       0.232804        0         1
166      3       0.232804        0         0
167      2       0.417910        0         1
168      3       0.232804        0         0
169      3       0.232804        0         0

[170 rows x 4 columns]
Accuracy: 0.6588235294117647


# Gaussian Bayes

## 手工

In [298]:
data = pd.DataFrame()
data['Height'] = [6,5.92,5.58,5.92,5,5.5,5.42,5.75]
data['Weight'] = [180,190,170,165,100,150,130,150]
data['Foot_Size'] = [12,11,12,10,6,8,7,9]
data['Gender'] = [0,0,0,0,1,1,1,1]

data

,Height,Weight,Foot_Size,Gender
0,6.00,180,12,0
1,5.92,190,11,0
2,5.58,170,12,0
3,5.92,165,10,0
4,5.00,100,6,1
5,5.50,150,8,1
6,5.42,130,7,1
7,5.75,150,9,1


In [299]:
def prob_density(std, mean, x):
    return 1 / np.sqrt(2 * np.pi * np.power(std, 2)) * np.exp(- np.power(x - mean, 2) / (2 * np.power(std, 2)))

In [300]:
p_c0 = np.mean(data['Gender'] == 0)
p_c1 = np.mean(data['Gender'] == 1)

p_c0, p_c1

(0.5, 0.5)

In [301]:
x1_c0_mean = np.mean(data['Height'][data['Gender']==0])
x1_c0_std = np.std(data['Height'][data['Gender']==0])

x2_c0_mean = np.mean(data['Weight'][data['Gender']==0])
x2_c0_std = np.std(data['Weight'][data['Gender']==0])

x3_c0_mean = np.mean(data['Foot_Size'][data['Gender']==0])
x3_c0_std = np.std(data['Foot_Size'][data['Gender']==0])

In [302]:
x1_c0_mean, x1_c0_std, x2_c0_mean, x2_c0_std, x3_c0_mean, x3_c0_std

(5.855, 0.16209565077447324, 176.25, 9.60143218483576, 11.25, 0.82915619758885)

In [303]:
x1_c1_mean = np.mean(data['Height'][data['Gender']==1])
x1_c1_std = np.std(data['Height'][data['Gender']==1])

x2_c1_mean = np.mean(data['Weight'][data['Gender']==1])
x2_c1_std = np.std(data['Weight'][data['Gender']==1])

x3_c1_mean = np.mean(data['Foot_Size'][data['Gender']==1])
x3_c1_std = np.std(data['Foot_Size'][data['Gender']==1])

In [304]:
x1_c1_mean, x1_c1_std, x2_c1_mean, x2_c1_std, x3_c1_mean, x3_c1_std

(5.4175, 0.2700347199898561, 132.5, 20.463381929681123, 7.5, 1.118033988749895)

In [305]:
from scipy.stats import norm

def prob(x):
    p1_c0 = norm(loc=x1_c0_mean, scale=x1_c0_std).cdf(x[0])
    p2_c0 = norm(loc=x2_c0_mean, scale=x2_c0_std).cdf(x[1])
    p3_c0 = norm(loc=x3_c0_mean, scale=x3_c0_std).cdf(x[2])
    
    p_c0_x = np.log(p_c0 * p1_c0 * p2_c0 * p3_c0)
    print('p_c0_x', p_c0_x)
    
    p1_c1 = norm(loc=x1_c1_mean, scale=x1_c1_std).cdf(x[0])
    p2_c1 = norm(loc=x2_c1_mean, scale=x2_c1_std).cdf(x[1])
    p3_c1 = norm(loc=x3_c1_mean, scale=x3_c1_std).cdf(x[2])
    
    p_c1_x = np.log(p_c1 * p1_c1 * p2_c1 * p3_c1)
    print('p_c1_x', p_c1_x)
    
    if p_c0_x > p_c1_x:
        return 0
    else:
        return 1
    

In [306]:
test_data = pd.DataFrame({'Height':[5.67,5],'Weight':[145,110],'Foot_Size':[12,7]})
test_data

,Height,Weight,Foot_Size
0,5.67,145,12
1,5.00,110,7


In [307]:
predict = []
for i in range(test_data.shape[0]):
    row = test_data[i:i+1]
    x = (row['Height'], row['Weight'], row['Foot_Size'])
    predict.append(prob(x)) 
predict

p_c0_x [-10.43390438]
p_c1_x [-1.20100105]
p_c0_x [-59.6193082]
p_c1_x [-6.60285434]


[1, 1]

In [308]:
test_results = pd.DataFrame({'Gender': predict})
print(test_data.join(test_results))

   Height  Weight  Foot_Size  Gender
0    5.67     145         12       1
1    5.00     110          7       1


## 自动

In [309]:
from sklearn.naive_bayes import GaussianNB

clf = GaussianNB()

X_train = data[['Height', 'Weight', 'Foot_Size']]
Y_train = [0,0,0,0,1,1,1,1]

In [310]:
clf.fit(X_train, Y_train)

GaussianNB()

In [311]:
print('Summary of parameters of our model: ')
print('\tClasses to predict: ', clf.classes_)
print('\tPrior on each class: ', clf.class_prior_)
print('\tMeans for each feature in each class:\n', clf.theta_)
print('\tStandard deviation for feature in each class: \n', clf.sigma_)

Summary of parameters of our model: 
	Classes to predict:  [0 1]
	Prior on each class:  [0.5 0.5]
	Means for each feature in each class:
 [[  5.855  176.25    11.25  ]
 [  5.4175 132.5      7.5   ]]
	Standard deviation for feature in each class: 
 [[2.62757340e-02 9.21875007e+01 6.87500734e-01]
 [7.29194840e-02 4.18750001e+02 1.25000073e+00]]


In [312]:
print('Some sample predictions:')
test_data = pd.DataFrame({'Height':[5.67,5],'Weight':[145,110],'Foot_Size':[12,7]})
test_results = pd.DataFrame({'gender': clf.predict(test_data)})
print(test_data.join(test_results))

Some sample predictions:
   Height  Weight  Foot_Size  gender
0    5.67     145         12       0
1    5.00     110          7       1
